# Sentiment Validation

In [ ]:
# df=pd.read_csv("/Users/niloofar/Downloads/Hotel_Reviews.csv")
# df1=df[df.Hotel_Name=="Britannia International Hotel Canary Wharf"]


# data_pos = df1['Positive_Review']
# data_neg = df1['Negative_Review']

In [ ]:

# from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
# import ast

In [1]:
import pandas as pd
import datetime
import re
import numpy as np
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.parsing.preprocessing import STOPWORDS


import nltk
from nltk.stem.porter import *
nltk.download('wordnet')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
from nltk.stem import WordNetLemmatizer, SnowballStemmer

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)



[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/niloofar/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
import numpy as np
import pandas as pd
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import spacy
from spacy.lemmatizer import Lemmatizer
from spacy.lang.en import English, STOP_WORDS
# from en_core_web_lg import *
# import en_core_web_lg

from tqdm import tqdm_notebook as tqdm
from pprint import pprint
import pickle

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import ast

In [3]:
def table_process(input_data):
    
    
    def sentiment_analyzer_scores(sentence):
        analyser = SentimentIntensityAnalyzer()
        score = analyser.polarity_scores(sentence)
        return("{}".format(str(score)))
   
    sentiment_eval=[sentiment_analyzer_scores(i) for i in input_data]
    sent_score=[ast.literal_eval(i) for i in  sentiment_eval]
    sent_table=pd.DataFrame(sent_score)
    sent_table['Document_No']=sent_table.index
    sent_table=sent_table[['compound','Document_No']]
#     table_sentiment=table_topic.merge(sent_table, on=['Document_No'])
#     table_sentiment.loc[table_sentiment.Topic_Name=="None" ,'compound']=0
    return(sent_table)
    

In [ ]:
# tp=pos_sent[pos_sent['compound']>0]
# fp=pos_sent[pos_sent['compound']<0]
# tn=neg_sent[neg_sent['compound']<0]
# fn=neg_sent[neg_sent['compound']>0]

In [ ]:
# recall=3483/(3483+1000)
# precision=3483/(3483+161)

In [4]:
import pickle
with open('/Users/niloofar/Documents/insight/HotelWatch/data/doc_pos', 'rb') as f:
            dd= pickle.load(f)


In [6]:
with open('/Users/niloofar/Documents/insight/HotelWatch/data/trip_sentence', 'rb') as f:
            df= pickle.load(f)

In [ ]:
# with open('/Users/niloofar/Documents/insight/HotelWatch/data/trip_sentence', 'rb') as f:
#             df= pickle.load(f)

In [204]:
# list(df.Review)

In [9]:
words = corpora.Dictionary(dd)
corpus = [words.doc2bow(doc) for doc in dd]
    

In [10]:
def compute_coherence_values(dictionary, corpus, texts, limit, start, step):

        coherence_values = []
        model_list = []
        for num_topics in range(start, limit, step):
#         model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
            model =gensim.models.ldamodel.LdaModel(corpus=corpus,
                                               id2word=dictionary,
                                               num_topics=num_topics,
                                               random_state=100,
                                               update_every=1,
                                               chunksize=100,
                                               passes=10,
                                               alpha='auto',
                                               per_word_topics=True)
            model_list.append(model)
            coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
            coherence_values.append(coherencemodel.get_coherence())

        return model_list, coherence_values
    

In [146]:
model_list, coherence_values = compute_coherence_values(dictionary=words, corpus=corpus, texts=dd, start=3, limit=7, step=1)



In [147]:
opt=coherence_values.index(max(coherence_values))
optimal_model=model_list[opt]

In [148]:
def format_topics_sentences(ldamodel, corpus, texts):
    
        sent_topics_df = pd.DataFrame()

        # Get main topic in each document
        for i, row_list in enumerate(ldamodel[corpus]):
            row = row_list[0] if ldamodel.per_word_topics else row_list            
            # print(row)
            row = sorted(row, key=lambda x: (x[1]), reverse=True)
            
            # Get the Dominant topic, Perc Contribution and Keywords for each document
            for j, (topic_num, prop_topic) in enumerate(row):
                if j == 0:  # => dominant topic
                    wp = ldamodel.show_topic(topic_num)
                    topic_keywords = ", ".join([word for word, prop in wp])
                    sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
                else:
                    break
        sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

        # Add original text to the end of the output
        contents = pd.Series(texts)
        sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
        return(sent_topics_df)

In [149]:
df_topic_sents_keywords = format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=list(df.Review))



In [150]:
final=df_topic_sents_keywords

In [151]:
final.columns=["Topic_num","Perc","Keywords","Text"]

In [210]:
topic_table=pd.DataFrame({'Topic_num':[0,1,2,3,4,5],"Topic":["Room_service","view","staff","location","Parking_charges","restaurant"]})

In [211]:
final=pd.merge(final,topic_table, on="Topic_num")

In [249]:
final.head()

,Topic_num,Perc,Keywords,Text,Topic,compound
0,0.0,0.2004,"room, bathroom, service, thing, corner, deal, ...","No, seriously, don't stay here.",Room_service,-0.4404
1,0.0,0.2364,"room, bathroom, service, thing, corner, deal, ...",Spend a few more dollars and stay nearby at a ...,Room_service,0.4404
2,0.0,0.2004,"room, bathroom, service, thing, corner, deal, ...",Only stayed overnight as we were joining a cru...,Room_service,0.0000
3,0.0,0.2004,"room, bathroom, service, thing, corner, deal, ...",We stayed on a Friday night one week & the nex...,Room_service,0.0000
4,0.0,0.2854,"room, bathroom, service, thing, corner, deal, ...",Both times were very similar in standards and ...,Room_service,-0.4023


In [206]:
def table_process(input_data):
    
    
    def sentiment_analyzer_scores(sentence):
        analyser = SentimentIntensityAnalyzer()
        score = analyser.polarity_scores(sentence)
        return("{}".format(str(score)))
   
    sentiment_eval=[sentiment_analyzer_scores(i) for i in input_data]
    sent_score=[ast.literal_eval(i) for i in  sentiment_eval]
    sent_table=pd.DataFrame(sent_score)
    sent_table['Document_No']=sent_table.index
    sent_table=sent_table[['compound','Document_No']]
#     table_sentiment=table_topic.merge(sent_table, on=['Document_No'])
#     table_sentiment.loc[table_sentiment.Topic_Name=="None" ,'compound']=0
    return(sent_table)
    

In [207]:
sent=table_process(final.Text)

In [217]:
final=pd.concat([final, sent['compound']], axis=1, ignore_index=False)

In [219]:
df.columns=[["Text","rating","Date"]]

In [226]:
df.reset_index(level=0, inplace=True)

In [244]:
finT=pd.concat([final[['compound','Topic',"Topic_num"]], df[["index","rating","Date"]]], axis=1, ignore_index=False)

In [192]:
tt1=list(tt['Text'])


In [195]:
318/len(tt1)

0.20476497102382485

In [109]:
192/412

0.46601941747572817

In [194]:
i=0
for word in tt1:
    if "restaurant" in word:
        i=i+1
print(i)
    

318


In [153]:
set(final.Keywords)

{'location, elevator, floor, minute, lobby, area, morning, marketplace, building, way',
 'restaurant, desk, port, food, shopping, lot, arena, bar, shop, water',
 'room, bathroom, service, thing, corner, deal, towel, shuttle, toilet, rate',
 'staff, place, people, breakfast, work, shower, bit, love, distance, luggage',
 'time, price, parking, door, experience, beach, car, sleep, friend, wall',
 'view, bed, day, downtown, book, street, bay, use, ship, trip'}

# dividing comment to sentence

In [250]:
import pickle
with open('/Users/niloofar/Documents/insight/data/cleaned/hotel2/doc_pos', 'rb') as f:
            dd= pickle.load(f)



In [253]:
import pickle
with open('/Users/niloofar/Documents/insight/data/cleaned/hotel2/trip_ad', 'rb') as f:
            trip_ad= pickle.load(f)



In [259]:
words = corpora.Dictionary(dd)
corpus = [words.doc2bow(doc) for doc in dd]

In [260]:
model_list, coherence_values = compute_coherence_values(dictionary=words, corpus=corpus, texts=dd, start=3, limit=15, step=1)



In [261]:
opt=coherence_values.index(max(coherence_values))
optimal_model=model_list[opt]

In [264]:
df_topic_sents_keywords = format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=list(trip_ad.Review))



In [270]:
dtopic=df_topic_sents_keywords

In [274]:
dtopic.columns=["Dominant_Topic","Perc","Keywords","Text"]

In [268]:
pd.Series(df_topic_sents_keywords.Dominant_Topic).value_counts()

8.0    1777
0.0     685
2.0     184
4.0      10
6.0       4
Name: Dominant_Topic, dtype: int64

In [297]:
tt=dtopic[dtopic['Dominant_Topic']==6]

In [298]:
tt

,Dominant_Topic,Perc,Keywords,Text
10,6.0,0.2899,"park, car, arena, block, towel, space, airline...",Hotel is well located Walking distance to BAYS...
552,6.0,0.2454,"park, car, arena, block, towel, space, airline...",This hotel is not only conveniently located bu...
845,6.0,0.2817,"park, car, arena, block, towel, space, airline...",Last time for this place! Drive 650 miles to h...
1152,6.0,0.2655,"park, car, arena, block, towel, space, airline...",Silvia Calvino the concierge was exceptionally...


In [299]:
tt1=list(tt['Text'])

i=0
for word in tt1:
    if "park" in word:
        i=i+1
print(i)
    

4


In [278]:
len(tt1)

685

In [293]:
(74+48)/len(tt1)

0.6630434782608695

In [ ]:
# i=0
# PK=[]
# for doc in df.Review:
       
#         nlp = spacy.load("en_core_web_sm")
#         nlp.pipe_names
#         tt = nlp(doc)
#         ses=[sent.text for sent in tt.sents]
#         PK.append(pd.DataFrame({"index":i,"Sent":ses}))
#         i=i+1
        


In [ ]:
# nlp = English()
# lp= spacy.load("en")
# nlp.Defaults.stop_words.update(stop_words)



In [ ]:
# def spacy_root(text):
#     nlp = spacy.load("en_core_web_sm")
#     doc=[]
#     l=[]
#     for word in text:
#         ss=nlp(word)
#         for chunk in ss.noun_chunks:
#             l.append(chunk.root.text)
#     doc.append(l)
#     return(doc)


# def lemmatizer(doc):
#         # This takes in a doc of tokens from the NER and lemmatizes them. 
#         # Pronouns (like "I" and "you" get lemmatized to '-PRON-', so I'm removing those.
#         doc = [token.lemma_ for token in doc if token.lemma_ != '-PRON-']
#         doc = u' '.join(doc)
#         return nlp.make_doc(doc)
    
    
# def remove_stopwords(doc):
#         spacy_nlp = spacy.load('en_core_web_sm')
#         spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS

#         for i in no_list:  
#             STOP_WORDS.add(i)

#         for word in STOP_WORDS:
#             lexeme = nlp.vocab[word]
#             lexeme.is_stop = True

#         tokens = [token.text for token in doc if not token.is_stop and token.is_punct != True and  len(token) >=3]
#         tokens=[i.lower() for i in tokens]
#         return tokens



In [ ]:
# import spacy

# [token.lemma_ for token in doc if token.lemma_ != '-PRON-']



# nlp = spacy.load("en_core_web_sm")
# doc = nlp(input_data)
# for token in doc:
# #     print([token  for token in doc if token!= '-PRON-'])
# #     print(token.text, token.pos_, token.dep_)
#       print(token.text)

In [ ]:
# import pandas as pd
# df = pd.concat(PK)

In [ ]:
topic1=topic[["Document_No","Dominant_Topic","Topic_Perc_Contrib","Keywords","Date","rating"]]

In [ ]:
df.columns=["Document_No","Review"]

In [ ]:
ss=pd.merge(df,topic1,on=["Document_No"])

In [ ]:
ss[["Keywords","Dominant_Topic"]].drop_duplicates()

In [ ]:
ss.drop_duplicates()

In [ ]:
df.set_index("Document_No", inplace = True)
trip_sentence = df.merge(trip_ad[["rating","Date"]], left_index=True, right_index=True)

In [ ]:
with open("/Users/niloofar/Documents/insight/HotelWatch/data/trip_ad", 'rb') as f:
            df= pickle.load(f)
        

In [ ]:
with open("/Users/niloofar/Documents/insight/HotelWatch/data/trip_sentence", 'wb') as f:
            pickle.dump(trip_sentence, f)
        

In [ ]:

no_list=['hotel','night','nothing','would','could','want','go','recommend','everything','be','was','good','ok','great','poor','miami','YVE',"friday","saturday",'monday','week'] 

In [ ]:
with open("/Users/niloofar/Documents/insight/HotelWatch/data/doc_pos", 'rb') as f:
            dd = pickle.load(f)

In [ ]:
import pickle
with open("/Users/niloofar/Documents/insight/HotelWatch/result/Topic", 'rb') as f:
            topic = pickle.load(f)



In [ ]:
topic1=topic.drop_duplicates()

In [ ]:
import pandas as pd

In [ ]:
pd.Series(ss['Dominant_Topic']).value_counts()

In [ ]:
trip_ad['Review'][422]

In [ ]:
set(topic1.Keywords)

In [ ]:
import pickle
with open('/Users/niloofar/Documents/insight/data/cleaned/hotel2/sentiment_topic_final', 'rb') as f:
            Topic= pickle.load(f)

In [ ]:
import pickle
with open('/Users/niloofar/Documents/insight/HotelWatch/result/Topic', 'rb') as f:
            Topic= pickle.load(f)

In [ ]:
Topic

In [ ]:
Topicf=Topic[["Document_No","Dominant_Topic","Keywords"]]

In [ ]:
Topicf.set_index("Document_No", inplace = True)
df1 = df.merge(Topicf, left_index=True, right_index=True)

In [ ]:
df1=df1.drop_duplicates()

In [ ]:
Topic[Topic['Dominant_Topic']==3]

In [ ]:
Topic['Text'][1426]

In [ ]:
DF2=pd.merge(df1[['rating','Date','Dominant_Topic','Keywords']], trip_ad, on=['Date','rating'])

In [ ]:
DF2=DF2.drop_duplicates()

In [ ]:
Topic['Keywords'][457]

In [ ]:
tdf=Topic[["Dominant_Topic","Keywords"]].drop_duplicates()

In [ ]:
tdf

In [ ]:
import pickle
with open('/Users/niloofar/Documents/insight/data/cleaned/hotel2/trip_ad', 'rb') as f:
            trip_ad= pickle.load(f)

In [ ]:
Topic.Keywords[2]

In [ ]:
dd[2]